The goal of this notebook is to verify I can load data from a different notebook.  Implement a simple model, and submit to the API.

In [1]:
import riiideducation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
env = riiideducation.make_env()

ModuleNotFoundError: No module named 'riiideducation.competition'

In a previous notebook, I read in and then pickled the data.  Lets see how fast i can load it.  Note, you need to add the other notebook to this project.

In [2]:
train_df = pd.read_parquet('/kaggle/input/00-make-pickle/train.10M.pq')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 8 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int16  
 3   content_type_id                 int8   
 4   task_container_id               int16  
 5   answered_correctly              int8   
 6   prior_question_elapsed_time     float32
 7   prior_question_had_explanation  boolean
dtypes: boolean(1), float32(1), int16(2), int32(1), int64(1), int8(2)
memory usage: 2.3 GB


Lets do a simple calculation to find average pct correct per question

In [3]:
full_df = train_df[train_df['content_type_id'] == 0]
question_averages_df = full_df[['content_id','answered_correctly']].groupby(['content_id']).agg(['sum','count'])
question_averages_df.columns = ['correct','total']
question_averages_df["pct_correct"] = question_averages_df['correct'] / question_averages_df['total']

In [4]:
question_averages_df.describe()

,correct,total,pct_correct
count,13523.000000,13523.000000,13523.000000
mean,4824.715448,7340.922872,0.709460
std,6419.482619,11455.927774,0.164486
min,0.000000,1.000000,0.000000
25%,1274.000000,1765.500000,0.609345
50%,3236.000000,4732.000000,0.734215
75%,5877.500000,8556.000000,0.835116
max,149422.000000,213605.000000,1.000000


Save out to file

In [5]:
# question_averages_df.to_csv('qa.csv')

In [6]:
# !ls

## Submit Code

In [7]:
## # You can only iterate through a result from `env.iter_test()` once
# so be careful not to lose it once you start iterating.
iter_test = env.iter_test()

In [8]:
# (test_df, sample_prediction_df) = next(iter_test)

In [9]:
# sample_prediction_df

In [10]:
# test_df

In [11]:
def make_prediction(test_df, question_averages_df) :
    # Join Tables ...
    prediction_df = test_df[['row_id','content_id','content_type_id']].merge(question_averages_df,left_on='content_id',right_on='content_id',how='left')
    # import numpy as np
    # prediction_df['random'] = np.random.rand(len(prediction_df))
    prediction_df = prediction_df.rename(columns={"pct_correct": "answered_correctly"})
    print("testdf len : {}".format(len(test_df)))
    print("DF len : {}".format(len(prediction_df)))
    return prediction_df[['row_id','answered_correctly','content_type_id']]

In [12]:
# Test Function
# prediction_df = make_prediction(test_df,question_averages_df)
# env.predict(prediction_df)

In [13]:
# prediction_df

In [14]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.loc[test_df.content_type_id == 0].reset_index(drop=True)
    prediction_df = make_prediction(test_df,question_averages_df)
    env.predict(prediction_df[['row_id', 'answered_correctly']])

testdf len : 18
DF len : 18
testdf len : 27
DF len : 27
testdf len : 26
DF len : 26
testdf len : 33
DF len : 33


In [15]:
prediction_df

,row_id,answered_correctly,content_type_id
0,74,0.626737,0
1,75,0.733943,0
2,76,0.412793,0
3,77,0.861466,0
4,78,0.455850,0
5,79,0.330109,0
6,80,0.809360,0
7,81,0.647611,0
8,82,0.435323,0
9,84,0.713415,0


In [16]:
test_df

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses
0,74,75311,275030867,8308,0,3,15000.0,False,"[1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, ...","[0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 3, 0, 0, ..."
1,75,31220886463,1305988022,396,0,4163,19000.0,True,NaN,NaN
2,76,48613916248,1310228392,11869,0,1458,26333.0,True,NaN,NaN
3,77,48613916248,1310228392,11871,0,1458,26333.0,True,NaN,NaN
4,78,48613916248,1310228392,11870,0,1458,26333.0,True,NaN,NaN
5,79,48613916248,1310228392,11872,0,1458,26333.0,True,NaN,NaN
6,80,48613916248,1310228392,11868,0,1458,26333.0,True,NaN,NaN
7,81,4693192735,1637273633,5935,0,3149,19000.0,True,NaN,NaN
8,82,1254131274,674533997,6000,0,1046,10000.0,True,NaN,NaN
9,84,69704234415,2093197291,12611,0,5448,28750.0,True,NaN,NaN
